In [7]:
from fastai.vision.all import load_learner, RandTransform, PILImage
from fastai.vision.all import *
import numpy as np
from pathlib import Path
import pathlib
from PIL import Image
import pandas as pd
import albumentations as A
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# Custom Albumentations transform (optional, not used here but retained)
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2

    def __init__(self, train_aug, valid_aug):
        store_attr()

    def before_call(self, b, split_idx):
        self.idx = split_idx

    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

# Cross-platform compatibility
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Load model
model_path = Path("mobilenet_student_distilled_1.pth")
if model_path.exists():
    learner = load_learner(model_path, cpu=True)
    print("Model loaded successfully")
else:
    raise FileNotFoundError(f"Model not found at {model_path}")

# Prediction function
def get_predictions(image_path, threshold=0.85):
    """
    Returns: (flag, recaptured_probability, pred_class)
    """
    pred_class, pred_idx, probabilities = learner.predict(image_path)
    recaptured_probability = probabilities[1]
    flag = 1 if recaptured_probability >= threshold else 0
    return flag, float(recaptured_probability), str(pred_class)

AttributeError: 'collections.OrderedDict' object has no attribute 'dls'

In [5]:
# Define base folder with images
base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder")

# Get all images (limit to first 5000)
image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))
# image_paths = image_paths[:5]

print(f"Found {len(image_paths)} images. Starting prediction..\n")

# Prediction loop
results = []

for img_path in image_paths:
    try:
        flag, probability, pred_class = get_predictions(img_path)
        results.append({
            'Image': str(img_path),
            'Class': pred_class,
            'Flag': flag,
            'Recaptured_Probability': probability
        })
    except Exception as e:
        print(f"Error with image {img_path.name}: {e}")

# Create DataFrame and save to CSV
df_results = pd.DataFrame(results)
print("\n Sample results:")
print(df_results.head())

Found 17 images. Starting prediction..




 Sample results:
                                                                                                                                                               Image  \
0  C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1708005082_RC_22012034837192_2025-04-10R1042025111233.jpeg   
1  C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1710004041_WC_22012035247470_2025-04-10R1042025023342.jpeg   
2        C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1722_WC_22012035192133_2025-04-10R1042025101043.jpeg   
3  C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\11_1710004041_WC_22012035247470_2025-04-10R1042025023353.jpeg   
4  C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\12_1715007030_WC_22012035242521_2025-04

In [6]:
df_results.head()

,Image,Class,Flag,Recaptured_Probability
0,C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1708005082_RC_22012034837192_2025-04-10R1042025111233.jpeg,clear,0,0.000064
1,C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1710004041_WC_22012035247470_2025-04-10R1042025023342.jpeg,clear,0,0.000255
2,C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\10_1722_WC_22012035192133_2025-04-10R1042025101043.jpeg,recaptured,1,0.997781
3,C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\11_1710004041_WC_22012035247470_2025-04-10R1042025023353.jpeg,clear,0,0.030299
4,C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder\12_1715007030_WC_22012035242521_2025-04-10R1042025111803.jpeg,clear,0,0.000999


In [7]:

df_results.to_csv("test.csv", index=False)
print("\n Results saved to 'test.csv'")


 Results saved to 'test.csv'


In [ ]:
import shutil
import os

# Define output folders
output_dir = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\flagged_nnms")
flag_0_dir = output_dir / "flag_0_A"
flag_1_dir = output_dir / "flag_1_A"

# Create directories if they don't exist
flag_0_dir.mkdir(parents=True, exist_ok=True)
flag_1_dir.mkdir(parents=True, exist_ok=True)

# Copy images to respective folders
for _, row in df_results.iterrows():
    src = Path(row['Image'])
    dest_dir = flag_1_dir if row['Flag'] == 1 else flag_0_dir
    try:
        shutil.copy(src, dest_dir)
    except Exception as e:
        print(f" Could not copy {src.name} to {dest_dir}: {e}")

print(f"\n Images have been copied into respective folders:\yu8/>\"n- {flag_0_dir}\n- {flag_1_dir}")
